In [1]:
import torch
from time import time
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision import transforms
from PIL import Image
import numpy as np
from torchvision.io import read_image
from matplotlib import image
from prepare_dataset import MyDataset,subset_generator
from utils import display_some_images,stats_of_image_and_label,MyCuda_Stats,get_output_features_of_model
from vgg_pretrained import vgg_all_freezed,total_trainable_parameters,full_vgg,vgg_layer_freeze
from Model import Network,new_model
from train_fn import train,validation,image_validation,validation_classes,train_batch
from tqdm import tqdm


In [3]:
fullvgg=full_vgg()
total_trainable_parameters(fullvgg)
total_trainable_parameters(fullvgg.features)
total_trainable_parameters(vgg_all_freezed())
total_trainable_parameters(vgg_layer_freeze(24))


Total trainable parameters of VGG is 	 138357544

Total trainable parameters of Sequential is 	 14714688

Total trainable parameters of Sequential is 	 0

Total trainable parameters of Sequential is 	 7079424


In [4]:
# DATA LOADER PARAMETERS
batch=16
CUDA_LAUNCH_BLOCKING=1 

In [5]:
MyCuda_Stats()

No of GPUs i have is 0


AssertionError: Torch not compiled with CUDA enabled

In [6]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
transform = transforms.Compose([transforms.ToTensor(),normalize])
target_transform=transforms.Compose([transforms.ToTensor()])

In [8]:
anno_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\train_set\annotations"
image_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\train_set\images"
train_data=MyDataset(anno_dir,image_dir,transform=transform, target_transform=None)

In [9]:
val_anno_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\val_set\annotations"
val_image_dir=r"C:\Users\Engineer Protobetz\Documents\Programs Directory\data\AffectNet\val_set\images"
val_data=MyDataset(val_anno_dir,val_image_dir,transform=transform, target_transform=None)

In [10]:
train_loader=torch.utils.data.DataLoader(train_data,
                                         batch_size=batch,
                                         shuffle=True,
                                         num_workers=4,
                                         pin_memory=True,
                                         sampler=None)
val_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=batch,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)
val_class_loader=torch.utils.data.DataLoader(val_data,
                                       batch_size=1,
                                       shuffle=False,
                                       num_workers=4,
                                       pin_memory=True,
                                      sampler=None)

In [11]:
print(len(train_loader))
print(len(val_loader))


17979
250


In [12]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [13]:
feature_extractor=vgg_layer_freeze(17)
total_trainable_parameters(feature_extractor)



Total trainable parameters of Sequential is 	 12979200


In [14]:

#input_features_for_denselayer=get_output_features_of_model(model=feature_extractor,batch_size=32,dataloader=train_loader)
input_features_for_denselayer=25088


In [15]:
model1=new_model(feature_extractor,input_features_for_denselayer).to(device)


In [16]:
total_trainable_parameters(model1)


Total trainable parameters of new_model is 	 115776520


In [17]:
weights=[0.74,0.537,0.91,0.949,0.976,0.985,0.912,0.9854]
loss1=nn.CrossEntropyLoss(weight=torch.tensor(weights).to(device))
loss_plain=nn.CrossEntropyLoss()
optimizer=optim.Adam(model1.parameters(),lr=0.0001)
classes=['Neutral','Happy','Sad','Surprise','Fear','Disgust','Anger','Contempt']

In [18]:
model1.load_state_dict(torch.load(r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\modelrecent.pth"))

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\DELL\\Desktop\\Python\\Affect_Net\\Affectnet\\models\\modelrecent.pth'

In [22]:
optimizer.load_state_dict(torch.load(r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\optimizer1.pth"))

In [1]:
torch.cuda.empty_cache()

NameError: name 'torch' is not defined

In [20]:
torch.cuda.memory_summary(device='cuda', abbreviated=True)

KeyError: 'allocated_bytes.all.current'

In [21]:
# NO OF EPOCHS IS 5
epochs=5
for epoch in range(epochs):
    print(f'Epoch-{epoch}')
    train_batch(train_loader,model1,loss1,optimizer)
    validation(val_loader,model1,loss1)


Epoch-0


  0%|                                                                             | 1/17979 [00:11<57:29:51, 11.51s/it]

loss:2.087878 batch:0/17979


  0%|                                                                            | 12/17979 [00:50<21:09:38,  4.24s/it]


KeyboardInterrupt: 

In [38]:
validation_classes(val_class_loader,model1,loss1,0)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:32<00:00, 121.71it/s]

test error-0.605083 
 Accuracy-78.000000%


In [39]:
validation_classes(val_class_loader,model1,loss1,1)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:29<00:00, 133.86it/s]

test error-0.258443 
 Accuracy-92.600000%


In [40]:
validation_classes(val_class_loader,model1,loss1,2)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:30<00:00, 130.65it/s]

test error-1.395475 
 Accuracy-59.000000%


In [41]:
validation_classes(val_class_loader,model1,loss1,3)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:29<00:00, 133.57it/s]

test error-2.490212 
 Accuracy-39.800000%


In [42]:
validation_classes(val_class_loader,model1,loss1,4)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:29<00:00, 134.20it/s]

test error-2.562142 
 Accuracy-42.600000%


In [43]:
validation_classes(val_class_loader,model1,loss1,5)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:29<00:00, 134.40it/s]

test error-3.990193 
 Accuracy-30.000000%


In [44]:
validation_classes(val_class_loader,model1,loss1,6)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:29<00:00, 133.90it/s]

test error-1.721756 
 Accuracy-52.200000%


In [45]:
validation_classes(val_class_loader,model1,loss1,7)

100%|█████████████████████████████████████████████████████████████████████████████| 3999/3999 [00:30<00:00, 131.81it/s]

test error-4.240774 
 Accuracy-7.214429%


In [48]:
torch.save(model1.state_dict(), r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\model(weightedloss1adam.pth")

In [49]:
torch.save(optimizer.state_dict(), r"C:\Users\DELL\Desktop\Python\Affect_Net\Affectnet\models\optimizer(weightedloss1adam)..pth")